<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/config/wavegan_tf112_py36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training Wavegan with tensorflow 1.12.0 and python 3.6.7

Official implementation of WaveGAN, a machine learning algorithm which learns to generate raw audio waveforms.

https://github.com/chrisdonahue/wavegan/

@inproceedings {donahue2019wavegan,
  title={Adversarial Audio Synthesis},
  author={Donahue, Chris and McAuley, Julian and Puckette, Miller},
  booktitle={ICLR},
  year={2019}
}

### Mount Google Drive
  In this notebook we mount our google drive so that we can store the output of the model while it trains.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Install recommended dependencies
    pip install tensorflow-gpu==1.12.0
    pip install scipy==1.0.0
    pip install matplotlib==3.0.2
    pip install librosa==0.6.2
    
    It's worth it to note that this code maybe compatible different versions of TensorFlow, however we follow the recommendations of the author.

In [6]:
!pip install tensorflow-gpu==1.12.0
!pip install scipy==1.0.0
!pip install matplotlib==3.0.2
!pip install librosa==0.6.2

## Install cuda9.0 for compatability

Here we download the CUDA toolkit from the Nvidia site, unpack it, add an apt-key to install the CUDA 9 libraires

In [4]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
print('add key: ok')
!sudo apt-get update
!sudo apt-get install cuda-libraries-9-0

--2019-04-25 23:57:29--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?1PoLcUgxTBMXhm6gUCDQcZiuzNhzzs2-l1vLgjYjdd8Zl844UfRye4DK8TQNb45B7SRhF0Y8avFqb2DF15i5Nzx8EcXr9K6x7vlvZvP_wqXsIx-sIVlo-U-azx3xD2Wv-kuOb7GrNWI8y2qQUyCgc3lL44w4yZNzdzeBbeHJEMzf5ckeBKo7ldT0_YH3x6RHRTukM4DRASSdtuHKxK1e [following]
--2019-04-25 23:57:30--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?1PoLcUgxTBMXhm6gUCDQcZiuzNhzzs2-l1vLgjYjdd8Zl844UfRye4DK8TQNb45B7SRhF0Y8avFqb2DF15i5Nzx8EcXr9K6x7vlvZvP_wqXsIx

## Clone wavegan, download unzip SC09 data

One of the main points of this notebook book is to train a model without having the data beforehand. In the next cell, we clone the repository that contains the model, download the data we want to train on and unzip the files in place to its own folder.

In [7]:
!git clone https://github.com/chrisdonahue/wavegan.git

Cloning into 'wavegan'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 74.27 MiB | 43.68 MiB/s, done.
Resolving deltas: 100% (325/325), done.


In [0]:
#!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz #about 60 seconds
#!tar -xzf /content/sc09.tar.gz #in /content
!rm -rf /content/bird2vec
!mkdir /content/bird2vec
!mkdir /content/bird2vec/train

!cp /content/gdrive/My\ Drive/from-nire/* \
   /content/bird2vec/train/

Before moving forward, we want to confirm that our cuda9 install is infact working. 
       - Note: At this point, the colab instance has both CUDA 9 and CUDA 10          installed. You can confirm this by opening the file explorer to the left. Following this path:
        ('>', Files> [^],>usr>local)
        The CUDA directories should be present. In any event, if CUDA 9 did not install properly, the next cell will throw an error.

In [9]:
# Confirm GPU is running
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
if len(get_available_gpus()) == 0:
    for i in range(4):
        print('WARNING: Not running on a GPU! See above for faster generation')
else:
    print('Training on GPU')

Training on GPU


### Start training

In the next cell the code to train the model will be executed. A few things to note:
        - The model is training directly from the '/content/' folder.
        - The output of the model will be stored in the afore linked google drive at ''/content/gdrive/My Drive/wgtrain.
        - The data is being read from '/content/SC09/train'
        
Since the data consists of small bits of audio, it is necessary to add additional flags:
        - 'data_first_slice' to signal small clips
        - 'data_fast_wav' to skip parsing with librosa and use scipy.
        
WARNING: If the data is a collection of MP3's do not use these flags
        
 Other flags:
        - 'wavegan_genr_pp' adds a post-processing filter to reduce noise
        - 'wavegan_dim' decreases the model size (to allow for faster training)
        - 'data_slice_rate' increase the length of the sample the model generates (16384 == 1 second, 65536 == 4 seconds)
        - 'train_summary_secs' increase the gap between checkpoints
        


In [0]:
!export CUDA_VISIBLE_DEVICES="0"
!python /content/wavegan/train_wavegan.py \
train \
/content/gdrive/My\ Drive/wgtrain-stream-kauai-6-2sec \
--data_dir /content/gdrive/My\ Drive/from-nire \
--data_fast_wav 
--data_slice_len 32768 \
--data_overlap_ratio 0.5 \
--data_pad_end \
--wavegan_dim 64 \
--wavegan_genr_pp \
--train_summary_secs 120

Found 18620 audio files in specified directory
--------------------------------------------------------------------------------
Generator vars
[100, 16384] (1638400): G/z_project/dense/kernel:0
[16384] (16384): G/z_project/dense/bias:0
[1, 25, 512, 1024] (13107200): G/upconv_0/conv2d_transpose/kernel:0
[512] (512): G/upconv_0/conv2d_transpose/bias:0
[1, 25, 256, 512] (3276800): G/upconv_1/conv2d_transpose/kernel:0
[256] (256): G/upconv_1/conv2d_transpose/bias:0
[1, 25, 128, 256] (819200): G/upconv_2/conv2d_transpose/kernel:0
[128] (128): G/upconv_2/conv2d_transpose/bias:0
[1, 25, 64, 128] (204800): G/upconv_3/conv2d_transpose/kernel:0
[64] (64): G/upconv_3/conv2d_transpose/bias:0
[1, 25, 1, 64] (1600): G/upconv_4/conv2d_transpose/kernel:0
[1] (1): G/upconv_4/conv2d_transpose/bias:0
[512, 1, 1] (512): G/pp_filt/conv1d/kernel:0
Total params: 19065857 (72.73 MB)
--------------------------------------------------------------------------------
Discriminator vars
[25, 1, 64] (1600): D/downco

The model is training. Please see the Tensorboard notebook for instructions on monitoring the training process